In [1]:
from google.colab import drive
import os
import zipfile
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/Master/2020_fall/Machine_learning/修課 - 吳沛遠/Final Project/"

Mounted at /content/drive
/content/drive/MyDrive/Master/2020_fall/Machine_learning/修課 - 吳沛遠/Final Project


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# Data Pre-processing:

In [3]:
import statsmodels.api as sm
from scipy.stats import t
from sklearn.impute import KNNImputer

def preprocess(data, train_idx):
    train = data[:train_idx]
    ## skewness of dependent variable
    train['SalePrice'] = np.log1p(train['SalePrice'])
    train_wo_nan = train[train.columns[~train.isnull().any()]]
    
    ## remove outlier by studentized residual
    conti_var = train_wo_nan[train_wo_nan.columns[train_wo_nan.dtypes!=np.object]].columns.tolist()[1:-1]
    x = train_wo_nan[conti_var]
    x = sm.add_constant(x)
    y = train_wo_nan['SalePrice']
    model = sm.OLS(y, x).fit() 
    # calculate studentized residuals 
    stud_res = model.outlier_test() 

    # define predictor variable values and studentized residuals 
    x = x.index.tolist()
    y = stud_res['student_resid']
    train_wo_out = train[(y<stats.t(df=len(y)-len(conti_var)-1).ppf((0.995))) & (y>stats.t(df=len(y)-len(conti_var)-1).ppf((0.005)))]
    
    # 完成training data remove outlier, 合併test data
    data_wo_out = pd.concat((train_wo_out, data[train_idx:]), sort=False).reset_index(drop=True)
    
    
    ## 補值
    data_wo_out["Alley"][data_wo_out["Alley"].isnull()]=0
    data_wo_out["Alley"][data_wo_out["Alley"]=='Grvl']=1
    data_wo_out["Alley"][data_wo_out["Alley"]=='Pave']=2
    data_wo_out["Alley"]=pd.to_numeric(data_wo_out["Alley"])

    data_wo_out['MasVnrType'][data_wo_out['MasVnrType'].isnull()] = 'CBlock'

    Bsmt_idx = data_wo_out[['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']].isnull().all(axis=1)
    qual = {'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1}
    for col in ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']:
      data_wo_out[col][Bsmt_idx] = 0
      for q in qual:
        data_wo_out[col][data_wo_out[col]==q] = qual[q]

    Exp_Qual = {'Gd':4, 'Av':3, 'Mn':2, 'No':0}
    for q in Exp_Qual:
      data_wo_out['BsmtExposure'][data_wo_out['BsmtExposure']==q] = Exp_Qual[q]

    Fin_Qual = {"GLQ":5,"ALQ":4,"BLQ":3,"Rec":2,"LwQ":1,"Unf":0}
    for q in Fin_Qual:
      data_wo_out['BsmtFinType1'][data_wo_out['BsmtFinType1']==q] = Fin_Qual[q]
      data_wo_out['BsmtFinType2'][data_wo_out['BsmtFinType2']==q] = Fin_Qual[q]

    for i in ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']:
      data_wo_out[i] = pd.to_numeric(data_wo_out[i])


    data_wo_out['FireplaceQu'][data_wo_out['Fireplaces']==0]=0
    for q in qual:
      data_wo_out['FireplaceQu'][data_wo_out['FireplaceQu']==q] = qual[q] 
    data_wo_out['FireplaceQu'] = pd.to_numeric(data_wo_out['FireplaceQu'])

    Garage_idx = data_wo_out[['GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond']].isnull().all(axis=1)
    for col in ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']:
      data_wo_out[col][Garage_idx] = 'None'
    data_wo_out['GarageYrBlt'] = data_wo_out['GarageYrBlt'].max()+1-data_wo_out['GarageYrBlt']
    data_wo_out['GarageYrBlt'][Garage_idx]=0
    data_wo_out['GarageYrBlt'] = pd.to_numeric(data_wo_out['GarageYrBlt'])


    data_wo_out['PoolQC'][data_wo_out['PoolArea']==0]=0
    for q in qual:
      data_wo_out['PoolQC'][data_wo_out['PoolQC']==q] = qual[q]  
    data_wo_out['PoolQC'] = pd.to_numeric(data_wo_out['PoolQC'])

    data_wo_out['Fence'][data_wo_out['Fence'].isnull()]=0
    FenceQual = {"GdPrv":4,"MnPrv":3,"GdWo":2,"MnWw":1}
    for q in FenceQual:
      data_wo_out['Fence'][data_wo_out['Fence']==q] = FenceQual[q]
    data_wo_out['Fence'] = pd.to_numeric(data_wo_out['Fence'])

    data_wo_out["MiscFeature"][data_wo_out["MiscFeature"].isnull()] = "None"

    ## several non consistent data filled up with mode or 0
    train_wo_out['BsmtExposure'][train_wo_out['BsmtExposure'].isnull()] = 0
    train_wo_out['BsmtFinType2'][train_wo_out['BsmtFinType2'].isnull()] = 0
    train_wo_out['Electrical'][train_wo_out['Electrical'].isnull()] = train_wo_out['Electrical'].mode().iloc[0]
    
    ## remove skewness
    cols = ['LotFrontage','LotArea', 'MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF',
            '1stFlrSF','2ndFlrSF','LowQualFinSF','GarageArea','WoodDeckSF',
            'OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','GrLivArea','PoolArea','MiscVal']
    for col in cols:
      data_wo_out[col]=np.log1p(data_wo_out[col])

    ## generate new variable
    data_wo_out = data_wo_out.assign(FullBath_n_2ndFlrSF = data_wo_out['FullBath'] * data_wo_out['2ndFlrSF'])
    data_wo_out = data_wo_out.assign(GarageCars_n_GarageArea = data_wo_out["GarageCars"] * data_wo_out["GarageArea"])

    # 'TotalBsmtSF','1stFlrSF', '2ndFlrSF', 'GrLivArea' these paramters has visible linear relation with log(SalesPrice+1); we transfer them to x**2
    cols = ['TotalBsmtSF','1stFlrSF', '2ndFlrSF', 'GrLivArea']
    for col in cols:
      data_wo_out[col] = data_wo_out[col]**2
    
    # one- hot encoding for all np.object columns
    data_dum = pd.get_dummies(data_wo_out)
    data_wo_out = pd.DataFrame(data_dum)

    # filled up Nans in numeric columns with KNN neighborhood 
    data_wo_out_n_y = data_wo_out.drop(["SalePrice"], axis=1)
    imputer = KNNImputer(n_neighbors=5) 
    imputed = imputer.fit_transform(data_wo_out_n_y) 
    data_wo_out_KNN = pd.DataFrame(imputed, columns=data_wo_out_n_y.columns) 
    y_train = data_wo_out[["SalePrice"]][data_wo_out["Id"]<=1460]
    x_train = data_wo_out_KNN[data_wo_out_KNN["Id"]<=1460]
    x_test = data_wo_out_KNN[data_wo_out_KNN["Id"]>1460]
    return x_train, y_train, x_test

In [4]:
# prep data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# concate train/test data
data = pd.concat((train, test), sort=False).reset_index(drop=True)
x_train, y_train, x_test = preprocess(data,1460)

In [5]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.3)

# Modeling:

Method Reference: https://medium.com/@permoonzz/kaggle-house-prices-advanced-regression-techniques-python-ensemble-learning%E5%AF%A6%E5%81%9A-99f757f4d326

## Rule based method

In [6]:
## Regression: multiple regression, Ridge regression、Lasso、Elastic net、Support vector regression

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
import math
import sklearn.model_selection as ms
import sklearn.metrics as sklm

In [7]:
from sklearn.linear_model import LinearRegression
Linear = LinearRegression(fit_intercept=True)

Linear_fit=Linear.fit(x_train,y_train)

y_linear_train=Linear_fit.predict(x_train)
y_linear_valid=Linear_fit.predict(x_valid)

# validation( train data and validate data)
print('RMSE_train_Linear = ' + str(math.sqrt(sklm.mean_squared_error(y_train, y_linear_train))))
print('RMSE_valid_Linear = ' + str(math.sqrt(sklm.mean_squared_error(y_valid, y_linear_valid))))

RMSE_train_Linear = 0.07428319495400032
RMSE_valid_Linear = 0.10600862867479989


In [8]:
from sklearn.linear_model import RidgeCV

alpha = [0.0001,0.001,0.01,0.1,1,10,100]
# find the best alpha and build model
Ridge = RidgeCV(cv=5, alphas=alpha) # the regressors X will be normalized before regression by subtracting the mean and dividing by the l2-norm.

Ridge_fit=Ridge.fit(x_train,y_train)

y_ridge_train=Ridge_fit.predict(x_train)
y_ridge_valid=Ridge_fit.predict(x_valid)

# validation( train data and validate data)
print('RMSE_train_Ridge = ' + str(math.sqrt(sklm.mean_squared_error(y_train, y_ridge_train))))
print('RMSE_valid_Ridge = ' + str(math.sqrt(sklm.mean_squared_error(y_valid, y_ridge_valid))))

RMSE_train_Ridge = 0.08006972558233426
RMSE_valid_Ridge = 0.09736388331325928


In [9]:
# LASSO:
from sklearn.linear_model import LassoCV
# set cross-validation alpha
alpha = [0.0001,0.001,0.01,0.1,1,10,100]
# find the best alpha and build model
Lasso = LassoCV(cv=5, alphas=alpha)
Lasso_fit=Lasso.fit(x_train,y_train)
y_lasso_train=Lasso_fit.predict(x_train)
y_lasso_valid=Lasso_fit.predict(x_valid)
# validation( train data and validate data)
print('RMSE_train_Lasso = ' + str(math.sqrt(sklm.mean_squared_error(y_train, y_lasso_train))))
print('RMSE_valid_Lasso = ' + str(math.sqrt(sklm.mean_squared_error(y_valid, y_lasso_valid))))

RMSE_train_Lasso = 0.08789150355407639
RMSE_valid_Lasso = 0.09730802550094862


In [10]:
from sklearn.linear_model import ElasticNetCV
# set cross-validation alpha and l1ratio
alpha=[0.0001,0.001,0.01,0.1,1,10,100]
l1ratio = [0.1, 0.5, 0.9, 0.95, 0.99, 1]
# find the best alpha/l1ratio and build model
elastic_cv = ElasticNetCV(cv=5,max_iter=int(1e7),alphas=alpha, l1_ratio=l1ratio)
elastic_fit = elastic_cv.fit(x_train, y_train)
y_el_train=elastic_fit.predict(x_train)
y_el_valid=elastic_fit.predict(x_valid)
# validation( train data and validate data)
print('RMSE_train_ElasticNet = ' + str(math.sqrt(sklm.mean_squared_error(y_train, y_el_train))))
print('RMSE_valid_ElasticNet = ' + str(math.sqrt(sklm.mean_squared_error(y_valid, y_el_valid))))

RMSE_train_ElasticNet = 0.0843504581764004
RMSE_valid_ElasticNet = 0.09762467208208711


In [11]:
from sklearn.svm import SVR
CV = {}
alpha=[1,10,100,1000,1e4,1e5]
for alp in alpha:
  svr = SVR(kernel='rbf', C=alp)
  svr_fit=svr.fit(x_train, y_train)
  y_svr_train=svr_fit.predict(x_train)
  y_svr_valid=svr_fit.predict(x_valid)
  # validation( train data and validate data)
  CV[alp] = math.sqrt(sklm.mean_squared_error(y_valid, y_svr_valid))

C = [key for key, cv in CV.items() if cv==min(CV.values())]
svr = SVR(kernel='rbf', C=C[0])
svr_fit=svr.fit(x_train, y_train)
y_svr_train=svr_fit.predict(x_train)
y_svr_valid=svr_fit.predict(x_valid)
print('C = ' + str(C[0]) + ', RMSE_train_svr = ' + str(math.sqrt(sklm.mean_squared_error(y_train, y_svr_train))))
print('C = ' + str(C[0]) + ', RMSE_valid_svr = ' + str(math.sqrt(sklm.mean_squared_error(y_valid, y_svr_valid))))

C = 100000.0, RMSE_train_svr = 0.07840236699955484
C = 100000.0, RMSE_valid_svr = 0.09959188072551768


## Boosting method: Adaboost、Gradient boosting、XGBoost、Staking、LightGBM

In [12]:
from lightgbm import LGBMRegressor
estimator=LGBMRegressor()
parameter_grid={'max_depth':range(2,5,1),
        'learning_rate':np.linspace(0.001,1,20),
        'feature_fraction':np.linspace(0.5,0.99,20),
        'bagging_fraction':np.linspace(0.1,0.99,20),
        'bagging_freq':range(5,10,1),
        'num_leaves':range(100,200,5),
        'min_data_in_leaf':range(50,200,10),
        'n_estimators':range(100,5000,100)}
grid = RandomizedSearchCV(estimator,parameter_grid,cv=3,scoring='neg_root_mean_squared_error',n_iter=int(5e2),n_jobs=-1)
grid.fit(x_train, y_train)
# print best parameter combination
print(grid.best_params_)

[LightGBM] [Warning] Unknown parameter: bagging_frequency
[LightGBM] [Warning] feature_fraction is set=0.7321052631578947, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7321052631578947
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] bagging_fraction is set=0.7089473684210525, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7089473684210525
{'num_leaves': 100, 'n_estimators': 700, 'min_data_in_leaf': 50, 'max_depth': 2, 'learning_rate': 0.1061578947368421, 'feature_fraction': 0.7321052631578947, 'bagging_frequency': 7, 'bagging_fraction': 0.7089473684210525}


In [13]:
lightgbm = LGBMRegressor(objective='regression', 
                         num_leaves=grid.best_params_["num_leaves"],
                         min_data_in_leaf=20,
                         learning_rate=grid.best_params_["learning_rate"],
                         n_estimators=grid.best_params_["n_estimators"],
                         metric="rmse",
                         max_depth=grid.best_params_["max_depth"],
                         bagging_fraction=grid.best_params_["bagging_fraction"],
                         bagging_freq=grid.best_params_["bagging_frequency"],
                         bagging_seed=9,
                         feature_fraction=grid.best_params_["feature_fraction"],
                         feature_fraction_seed=7,
                         verbose=-1)
lgbm_fit = lightgbm.fit(x_train, y_train)
y_lgbm_train=lgbm_fit.predict(x_train)
y_lgbm_valid=lgbm_fit.predict(x_valid)
print('RMSE_train_LGBM = ' + str(math.sqrt(sklm.mean_squared_error(y_train, y_lgbm_train))))
print('RMSE_valid_LGBM = ' + str(math.sqrt(sklm.mean_squared_error(y_valid, y_lgbm_valid))))

RMSE_train_LGBM = 0.05054890078784122
RMSE_valid_LGBM = 0.09996264805327666


In [14]:
from xgboost.sklearn import XGBRegressor
estimator=XGBRegressor()
parameter_grid={
        'max_depth':range(2,8,1),
        'learning_rate':np.linspace(0.001,1,20),
        'colsample_bytree':np.linspace(0.1,0.99,20),
        'n_estimators':range(100,5000,100),
        'min_child_weight':np.linspace(1,5,20),
        'gamma':np.linspace(0.1,0.99,20),
        'subsample':np.linspace(0.1,0.9,20),
        'reg_alpha':np.linspace(0.01,3,20),
        'reg_lambda':np.linspace(0.01,3,20)}
grid = RandomizedSearchCV(estimator,parameter_grid,cv=3,scoring = 'neg_root_mean_squared_error',n_iter=int(5e2),n_jobs=-1)
grid.fit(x_train, y_train)
# print best parameter combination
print (grid.best_params_)

{'subsample': 0.3526315789473684, 'reg_lambda': 0.9542105263157896, 'reg_alpha': 0.1673684210526316, 'n_estimators': 1700, 'min_child_weight': 3.7368421052631575, 'max_depth': 5, 'learning_rate': 0.2113157894736842, 'gamma': 0.1, 'colsample_bytree': 0.5684210526315789}


In [15]:
xgb= XGBRegressor(base_score=0.5, 
          booster='gbtree', 
          colsample_bylevel=1,
          colsample_bynode=1, 
          colsample_bytree=grid.best_params_["colsample_bytree"], 
          gamma=grid.best_params_["gamma"],
          importance_type='gain', 
          learning_rate=grid.best_params_["learning_rate"],
          max_delta_step=0,
          max_depth=grid.best_params_["max_depth"], 
          missing=1, 
          min_child_weight=grid.best_params_["min_child_weight"],
          n_estimators=grid.best_params_["n_estimators"],
          n_jobs=1, 
          nthread=None, 
          objective='reg:squarederror', 
          random_state=0,
          reg_lambda=grid.best_params_["reg_lambda"],
          reg_alpha=grid.best_params_["reg_alpha"],
          scale_pos_weight=1, 
          seed=None,
          silent=None, 
          subsample=grid.best_params_["subsample"], 
          verbosity=1)
xgb_fit = xgb.fit(x_train, y_train)
y_xgb_train=xgb_fit.predict(x_train)
y_xgb_valid=xgb_fit.predict(x_valid)
print('RMSE_train_XGB = ' + str(math.sqrt(sklm.mean_squared_error(y_train, y_xgb_train))))
print('RMSE_valid_XGB = ' + str(math.sqrt(sklm.mean_squared_error(y_valid, y_xgb_valid))))

RMSE_train_XGB = 0.08070159787335034
RMSE_valid_XGB = 0.11283729769354162


# Voting, Stacking, Blending

In [16]:
from sklearn.ensemble import VotingRegressor
# Build Model
vote_mod = VotingRegressor([('Ridge', Ridge_fit), ('Lasso',Lasso_fit),('Elastic',elastic_fit),('lgbm',lgbm_fit),('xgb',xgb_fit)])
vote = vote_mod.fit(x_train, y_train.values.ravel())
# predict train/test y
vote_pred_train=vote.predict(x_train)
vote_pred_valid=vote.predict(x_valid)
# validation( train data and validate data)
print('RMSE_train_Voting = ' + str(math.sqrt(sklm.mean_squared_error(y_train, vote_pred_train))))
print('RMSE_valid_Voting = ' + str(math.sqrt(sklm.mean_squared_error(y_valid, vote_pred_valid))))

[LightGBM] [Warning] feature_fraction is set=0.7321052631578947, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7321052631578947
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] bagging_fraction is set=0.7089473684210525, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7089473684210525
RMSE_train_Voting = 0.0711907344351661
RMSE_valid_Voting = 0.09379223744549266


In [ ]:
from mlxtend.regressor import StackingRegressor
from sklearn.ensemble import GradientBoostingRegressor
# Build Model
gbdt=GradientBoostingRegressor(learning_rate=0.05,
                max_leaf_nodes=3,
                n_estimators=100)
stregr = StackingRegressor(regressors=[Ridge_fit, Lasso_fit, elastic_fit, lgbm_fit,xgb_fit], 
                           meta_regressor=gbdt, 
                           use_features_in_secondary=True)
stack_mod=stregr.fit(x_train, y_train.values.ravel())
# predict train/test y
stacking_pred_train=stack_mod.predict(x_train)
stacking_pred_valid=stack_mod.predict(x_valid)
# validation( train data and validate data)
print('RMSE_train_Stacking = ' + str(math.sqrt(sklm.mean_squared_error(y_train, stacking_pred_train))))
print('RMSE_valid_Stacking = ' + str(math.sqrt(sklm.mean_squared_error(y_valid, stacking_pred_valid))))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-17-d74fba26e0ae>", line 1, in <cell line: 1>
    from mlxtend.regressor import StackingRegressor
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1002, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 945, in _find_spec
  File "<frozen importlib._bootstrap_external>", line 1439, in find_spec
  File "<frozen importlib._bootstrap_external>", line 1408, in _get_spec
  File "<frozen importlib._bootstrap_external>", line 1366, in _path_importer_cache
OSError: [Errno 107] Transport endpoint is not connected

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py",

In [ ]:
# set weight list
weight=list(np.linspace(0.1,1,91))
# create outcome dataframe 
train_mse=[]
valid_mse=[]
for i in weight:
    blending_pred_train=(i*vote_pred_train)+((1-i)*stacking_pred_train)
    blending_pred_valid=(i*vote_pred_valid)+((1-i)*stacking_pred_valid)
    train_mse.append(math.sqrt(sklm.mean_squared_error(y_train, blending_pred_train)))
    valid_mse.append(math.sqrt(sklm.mean_squared_error(y_valid, blending_pred_valid)))
blending_output=pd.DataFrame({'weight':weight,
                       'train_mse':train_mse,
                       'valid_mse':valid_mse})
# print top 10 weight value
weight = blending_output.sort_values(by=['valid_mse'],ascending=True)
print(weight.head(10))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-17-d9dddf9dc5af>", line 7, in <cell line: 6>
    blending_pred_train=(i*vote_pred_train)+((1-i)*stacking_pred_train)
NameError: name 'stacking_pred_train' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  Fi

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



  File "/usr/lib/python3.10/posixpath.py", line 384, in abspath
    cwd = os.getcwd()
OSError: [Errno 107] Transport endpoint is not connected
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-17-d9dddf9dc5af>", line 7, in <cell line: 6>
    blending_pred_train=(i*vote_pred_train)+((1-i)*stacking_pred_train)
NameError: name 'stacking_pred_train' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist

# Submit the predict data

In [ ]:
# predict and blending
final_pred_voting=vote.predict(x_test)
final_pred_stacking=stack_mod.predict(x_test)
w = weight.iloc[0]["weight"]
final_pred_blending=w*final_pred_voting+(1-w)*final_pred_stacking
# reverse log y
final_pred_blending=np.expm1(final_pred_blending)
a=list(range(1461,2920,1))
submit=pd.DataFrame({'Id':list(range(1461,2920,1)),
                     'SalePrice':final_pred_blending})
    
# save as csv     
submit.to_csv('submit.csv',index=False)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-17-0dcd49c6e20c>", line 3, in <cell line: 3>
    final_pred_stacking=stack_mod.predict(x_test)
NameError: name 'stack_mod' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python3.10/di